# Agent V5: The Full Agent

A monolithic agent with unified task orchestration. All sub-agent work flows through the
`TaskBroker`, giving us concurrent execution, progress events, and cancellation.

Extends the Coordinator agent with async task tools:
- `delegate(agent_name, prompt)` -- synchronous delegation (submit + wait)
- `submit_task(agent_name, prompt)` -- fire-and-forget, returns task_id
- `check_tasks()` -- poll status and results of all background tasks

Between turns, completed background tasks are automatically injected into the prompt.

**Prerequisite:** Run `scripts/db_ingest_bookstore_sqlite.sh` to create the bookstore database.

In [ ]:
from agentic_patterns.core.agents import AgentSpec, OrchestratorAgent
from agentic_patterns.core.config.config import PROMPTS_DIR
from agentic_patterns.core.workspace import clean_up_session
from agentic_patterns.tools.file import get_all_tools as get_file_tools
from agentic_patterns.tools.sandbox import get_all_tools as get_sandbox_tools
from agentic_patterns.tools.todo import get_all_tools as get_todo_tools
from agentic_patterns.tools.format_conversion import (
    get_all_tools as get_format_conversion_tools,
)
from agentic_patterns.agents.data_analysis import get_spec as get_data_analysis_spec
from agentic_patterns.agents.sql import get_spec as get_sql_spec
from agentic_patterns.agents.vocabulary import get_spec as get_vocabulary_spec

## Agent Definition

The full agent extends the Coordinator with async task support. The `OrchestratorAgent`
now creates a `TaskBroker` internally when `sub_agents` are present, and auto-adds
`delegate`, `submit_task`, and `check_tasks` tools.

In [ ]:
spec = AgentSpec(
    name="full_agent",
    system_prompt_path=PROMPTS_DIR / "the_complete_agent" / "agent_full.md",
    tools=(
        get_file_tools()
        + get_sandbox_tools()
        + get_todo_tools()
        + get_format_conversion_tools()
    ),
    sub_agents=[
        get_data_analysis_spec(),
        get_sql_spec(),
        get_vocabulary_spec(),
    ],
)
print(spec)

In [ ]:
clean_up_session()
agent = OrchestratorAgent(spec, verbose=True)

## Turn 1: Synchronous Delegation

Ask about data in the bookstore database. The agent should use `delegate` (synchronous)
since it needs the result immediately to continue.

In [ ]:
async with agent:
    prompt_1 = """Query the bookstore database: how many books are there,
what genres are represented, and what is the average price per genre?
Save the per-genre results to a CSV file."""

    result_1 = await agent.run(prompt_1)

    print("\n--- Agent Output ---")
    print(result_1.output)

## Turn 2: Parallel Background Tasks

Ask the agent to do multiple independent tasks. It should use `submit_task` to run
them in parallel (SQL query and data visualization), then `check_tasks` to collect results,
and finally assemble a report.

In [ ]:
async with agent:
    prompt_2 = """I need two things done in parallel:
1. Query the bookstore database for the top 5 most expensive books with their authors and genres.
2. Create a bar chart showing the average price per genre (save as /workspace/avg_price_by_genre.png).

Submit both as background tasks, then check on them. Once both are done,
write a markdown report at /workspace/bookstore_report.md combining all findings."""

    result_2 = await agent.run(prompt_2)

    print("\n--- Agent Output ---")
    print(result_2.output)

## Verify Workspace

In [ ]:
from agentic_patterns.core.workspace import list_workspace_files, read_from_workspace

for path in sorted(list_workspace_files("*")):
    try:
        content = read_from_workspace(path)
    except UnicodeDecodeError:
        content = "<binary file>"
    print(f"--- {path} ---")
    print(content)
    print()